In [1]:
import os as os

import numpy as np

import tensorflow.keras as keras
import tensorflow.train as train
import tensorflow.nn as nn

from sklearn.model_selection import train_test_split

In [2]:
def convertLabel(sequence): #gives a nb to the aminoacid
    label = sequence[7]
    aminoacidList = ["ALA", "ARG", "ASN", "ASP", "CYS", "GLU", "GLN", "GLY", "HIS", "ILE",
                    "LEU", "LYS", "MET", "PHE", "PRO", "PYL", "SEL", "SER", "THR", "TRP", "TYR", "VAL"]
    for i in range (0, 22):
        if label == aminoacidList[i]:
            return i

In [3]:
def load_data():


    labelList = []

    with open('matrix.csv') as f:

        nbOfSamples = int(f.readline())
        i = 0

        shapes = f.readline() # length of sequence + nb of features
        input_shape = int(shapes.split(",")[1])


        distancesList = np.zeros((nbOfSamples, input_shape))
        labelList = np.zeros(nbOfSamples)


        while i < nbOfSamples:
            # Extracting labels
            sequence = f.readline().split(",")
            labelList[i] = convertLabel(sequence)

            # Extracting distances
            distancesString = f.readline()
            distancesArray = np.array([float(s) for s in distancesString.split(",")])
            distancesList[i, :] = distancesArray

            # empty line
            f.readline()

            i += 1
    return input_shape, distancesList, labelList

In [4]:
# returns a short sequential model
def create_model():
    # activation function: leaky ReLU
    leakyrelu = lambda x: keras.activations.relu(x, alpha=0.01, max_value=None, threshold=0)

    # creation du reseau de neurones
    model = keras.models.Sequential([

        # hidden layer
        keras.layers.Dense(units = 500, activation = leakyrelu),
        keras.layers.Dropout(rate=0.2),
        keras.layers.Dense(units = 300, activation = leakyrelu),
        keras.layers.Dropout(rate=0.2),
        keras.layers.Dense(units = 200, activation = leakyrelu),
        keras.layers.Dropout(rate=0.2),
        keras.layers.Dense(units = 100, activation = leakyrelu),
        keras.layers.Dropout(rate=0.2),
        keras.layers.Dense(units = 75, activation = leakyrelu),
        keras.layers.Dropout(rate=0.2),
        keras.layers.Dense(units = 53, activation = leakyrelu),
        keras.layers.Dropout(rate=0.2),

        # final layer
        keras.layers.Dense(units = 22, activation = nn.softmax),
    ])

    model.compile(optimizer=keras.optimizers.Adam(),
                  loss=keras.losses.sparse_categorical_crossentropy,
                  metrics=['accuracy'])

    return model

In [5]:
# Data processing
input_shape, distances, labels = load_data()


x_train, x_test, y_train, y_test = train_test_split(
    distances, labels, test_size=0.2, random_state=42)

In [6]:
# Create checkppoints

checkpoint_path = "checkpoints/checkpoint.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = keras.callbacks.ModelCheckpoint(
    checkpoint_path, save_weights_only=True, verbose=1,
    # save weights, every 5-epoch
    period=5)

latest = train.latest_checkpoint(checkpoint_dir)

In [7]:
# create a model instance

model = create_model()

if latest: model.load_weights(latest)

Instructions for updating:
Colocations handled automatically by placer.


In [8]:
# training
model.fit(x_train, y_train, epochs = 500,
          validation_data = (x_test, y_test),
          callbacks = [cp_callback]) # save checkpoints at the end of each epoch

model.save('my_model.h5')

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Train on 167 samples, validate on 42 samples
Epoch 1/500
167/167 [==============================] - 1s 5ms/sample - loss: 4.4735 - acc: 0.0359 - val_loss: 2.9529 - val_acc: 0.0714
Epoch 2/500
167/167 [==============================] - 0s 399us/sample - loss: 3.2461 - acc: 0.0778 - val_loss: 2.9835 - val_acc: 0.1429
Epoch 3/500
167/167 [==============================] - 0s 396us/sample - loss: 3.2635 - acc: 0.0659 - val_loss: 2.9893 - val_acc: 0.0714
Epoch 4/500
167/167 [==============================] - 0s 392us/sample - loss: 3.1299 - acc: 0.1138 - val_loss: 3.0110 - val_acc: 0.0952
Epoch 5/500
 32/167 [====>.........................] - ETA: 0s - loss: 3.1480 - acc: 0.0625
Epoch 00005: saving model to checkpoints/checkpoint.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
Instructions for updating:
Use tf.train.CheckpointManager to manage checkpoints rather th

167/167 [==============================] - 0s 401us/sample - loss: 2.8992 - acc: 0.1018 - val_loss: 2.9360 - val_acc: 0.1190
Epoch 32/500
167/167 [==============================] - 0s 399us/sample - loss: 2.9022 - acc: 0.1317 - val_loss: 2.9471 - val_acc: 0.1190
Epoch 33/500
167/167 [==============================] - 0s 343us/sample - loss: 2.8540 - acc: 0.1377 - val_loss: 2.9336 - val_acc: 0.1190
Epoch 34/500
167/167 [==============================] - 0s 334us/sample - loss: 2.9032 - acc: 0.1138 - val_loss: 2.9186 - val_acc: 0.1190
Epoch 35/500
 32/167 [====>.........................] - ETA: 0s - loss: 2.9076 - acc: 0.0625
Epoch 00035: saving model to checkpoints/checkpoint.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
167/167 [==============================] - 0s 493us/sample - loss: 2.9231 - acc: 0.0898 - val_loss: 2.9308 - val_acc: 0.1667
Epoch 36/500
167/167 [==============================] - 0s 358us/sample - loss: 2.8687 - acc: 0.1257 - val_loss: 2.9297 - val_acc:

167/167 [==============================] - 0s 500us/sample - loss: 2.8396 - acc: 0.1138 - val_loss: 2.9461 - val_acc: 0.1190
Epoch 66/500
167/167 [==============================] - 0s 355us/sample - loss: 2.7680 - acc: 0.1317 - val_loss: 2.9562 - val_acc: 0.1190
Epoch 67/500
167/167 [==============================] - 0s 345us/sample - loss: 2.8917 - acc: 0.1377 - val_loss: 2.9243 - val_acc: 0.1190
Epoch 68/500
167/167 [==============================] - 0s 365us/sample - loss: 2.7764 - acc: 0.1317 - val_loss: 2.9244 - val_acc: 0.0238
Epoch 69/500
167/167 [==============================] - 0s 355us/sample - loss: 2.8492 - acc: 0.1377 - val_loss: 2.9161 - val_acc: 0.0476
Epoch 70/500
 32/167 [====>.........................] - ETA: 0s - loss: 2.8459 - acc: 0.0312
Epoch 00070: saving model to checkpoints/checkpoint.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
167/167 [==============================] - 0s 504us/sample - loss: 2.8116 - acc: 0.0898 - val_loss: 2.9546 - val_acc:

167/167 [==============================] - 0s 520us/sample - loss: 2.8011 - acc: 0.1257 - val_loss: 2.8960 - val_acc: 0.0952
Epoch 101/500
167/167 [==============================] - 0s 352us/sample - loss: 2.7905 - acc: 0.1138 - val_loss: 2.9514 - val_acc: 0.0952
Epoch 102/500
167/167 [==============================] - 0s 342us/sample - loss: 2.7588 - acc: 0.1437 - val_loss: 2.9950 - val_acc: 0.1190
Epoch 103/500
167/167 [==============================] - 0s 345us/sample - loss: 2.8680 - acc: 0.1317 - val_loss: 2.9308 - val_acc: 0.1190
Epoch 104/500
167/167 [==============================] - 0s 358us/sample - loss: 2.8101 - acc: 0.1078 - val_loss: 2.9081 - val_acc: 0.0476
Epoch 105/500
 32/167 [====>.........................] - ETA: 0s - loss: 2.8586 - acc: 0.1250
Epoch 00105: saving model to checkpoints/checkpoint.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
167/167 [==============================] - 0s 487us/sample - loss: 2.8091 - acc: 0.1078 - val_loss: 2.9213 - val

167/167 [==============================] - 0s 498us/sample - loss: 2.6990 - acc: 0.1377 - val_loss: 2.9472 - val_acc: 0.1429
Epoch 136/500
167/167 [==============================] - 0s 372us/sample - loss: 2.7395 - acc: 0.1377 - val_loss: 2.9599 - val_acc: 0.1190
Epoch 137/500
167/167 [==============================] - 0s 465us/sample - loss: 2.7224 - acc: 0.1257 - val_loss: 2.9275 - val_acc: 0.1190
Epoch 138/500
167/167 [==============================] - 0s 488us/sample - loss: 2.7223 - acc: 0.1078 - val_loss: 2.9084 - val_acc: 0.1190
Epoch 139/500
167/167 [==============================] - 0s 468us/sample - loss: 2.7339 - acc: 0.1497 - val_loss: 2.9040 - val_acc: 0.1190
Epoch 140/500
 32/167 [====>.........................] - ETA: 0s - loss: 2.7129 - acc: 0.0938
Epoch 00140: saving model to checkpoints/checkpoint.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
167/167 [==============================] - 0s 544us/sample - loss: 2.7445 - acc: 0.1138 - val_loss: 2.9058 - val

...

167/167 [==============================] - 0s 700us/sample - loss: 2.4365 - acc: 0.1737 - val_loss: 3.3733 - val_acc: 0.0476
Epoch 346/500
167/167 [==============================] - 0s 516us/sample - loss: 2.3894 - acc: 0.1677 - val_loss: 3.4658 - val_acc: 0.1190
Epoch 347/500
167/167 [==============================] - 0s 512us/sample - loss: 2.4011 - acc: 0.2275 - val_loss: 3.4153 - val_acc: 0.1190
Epoch 348/500
167/167 [==============================] - 0s 494us/sample - loss: 2.3789 - acc: 0.2156 - val_loss: 3.4693 - val_acc: 0.0714
Epoch 349/500
167/167 [==============================] - 0s 492us/sample - loss: 2.4105 - acc: 0.2096 - val_loss: 3.5657 - val_acc: 0.0952
Epoch 350/500
 32/167 [====>.........................] - ETA: 0s - loss: 2.3103 - acc: 0.3438
Epoch 00350: saving model to checkpoints/checkpoint.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
167/167 [==============================] - 0s 578us/sample - loss: 2.3543 - acc: 0.2216 - val_loss: 3.8003 - val

167/167 [==============================] - 0s 573us/sample - loss: 2.3252 - acc: 0.2275 - val_loss: 3.8025 - val_acc: 0.0952
Epoch 381/500
167/167 [==============================] - 0s 400us/sample - loss: 2.3235 - acc: 0.2156 - val_loss: 3.6380 - val_acc: 0.1429
Epoch 382/500
167/167 [==============================] - 0s 386us/sample - loss: 2.3231 - acc: 0.2156 - val_loss: 3.7202 - val_acc: 0.1429
Epoch 383/500
167/167 [==============================] - 0s 429us/sample - loss: 2.3272 - acc: 0.2395 - val_loss: 3.8152 - val_acc: 0.0952
Epoch 384/500
167/167 [==============================] - 0s 470us/sample - loss: 2.3906 - acc: 0.2156 - val_loss: 3.5893 - val_acc: 0.1429
Epoch 385/500
 32/167 [====>.........................] - ETA: 0s - loss: 2.3912 - acc: 0.2188
Epoch 00385: saving model to checkpoints/checkpoint.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
167/167 [==============================] - 0s 611us/sample - loss: 2.4459 - acc: 0.1916 - val_loss: 3.7494 - val

167/167 [==============================] - 0s 527us/sample - loss: 2.2582 - acc: 0.2455 - val_loss: 3.9904 - val_acc: 0.2143
Epoch 416/500
167/167 [==============================] - 0s 368us/sample - loss: 2.2613 - acc: 0.2575 - val_loss: 4.0129 - val_acc: 0.2619
Epoch 417/500
167/167 [==============================] - 0s 363us/sample - loss: 2.2692 - acc: 0.2096 - val_loss: 4.2898 - val_acc: 0.1905
Epoch 418/500
167/167 [==============================] - 0s 359us/sample - loss: 2.3161 - acc: 0.2096 - val_loss: 4.3378 - val_acc: 0.1429
Epoch 419/500
167/167 [==============================] - 0s 372us/sample - loss: 2.3473 - acc: 0.1856 - val_loss: 3.7791 - val_acc: 0.2143
Epoch 420/500
 32/167 [====>.........................] - ETA: 0s - loss: 2.0649 - acc: 0.3750
Epoch 00420: saving model to checkpoints/checkpoint.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
167/167 [==============================] - 0s 511us/sample - loss: 2.2632 - acc: 0.2156 - val_loss: 4.1552 - val

167/167 [==============================] - 0s 570us/sample - loss: 2.2874 - acc: 0.2096 - val_loss: 4.0594 - val_acc: 0.1905
Epoch 451/500
167/167 [==============================] - 0s 421us/sample - loss: 2.2740 - acc: 0.2096 - val_loss: 4.3681 - val_acc: 0.1190
Epoch 452/500
167/167 [==============================] - 0s 406us/sample - loss: 2.1886 - acc: 0.2515 - val_loss: 4.4411 - val_acc: 0.1429
Epoch 453/500
167/167 [==============================] - 0s 407us/sample - loss: 2.2263 - acc: 0.2216 - val_loss: 4.4598 - val_acc: 0.1429
Epoch 454/500
167/167 [==============================] - 0s 397us/sample - loss: 2.2991 - acc: 0.2455 - val_loss: 4.1237 - val_acc: 0.1667
Epoch 455/500
 32/167 [====>.........................] - ETA: 0s - loss: 2.3860 - acc: 0.2500
Epoch 00455: saving model to checkpoints/checkpoint.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
167/167 [==============================] - 0s 486us/sample - loss: 2.2952 - acc: 0.2635 - val_loss: 4.2086 - val

167/167 [==============================] - 0s 700us/sample - loss: 2.2614 - acc: 0.2575 - val_loss: 4.1049 - val_acc: 0.1905
Epoch 486/500
167/167 [==============================] - 0s 506us/sample - loss: 2.1872 - acc: 0.2216 - val_loss: 3.9772 - val_acc: 0.1667
Epoch 487/500
167/167 [==============================] - 0s 416us/sample - loss: 2.2216 - acc: 0.2395 - val_loss: 4.0804 - val_acc: 0.1667
Epoch 488/500
167/167 [==============================] - 0s 401us/sample - loss: 2.1491 - acc: 0.2575 - val_loss: 4.3746 - val_acc: 0.1429
Epoch 489/500
167/167 [==============================] - 0s 395us/sample - loss: 2.1134 - acc: 0.2814 - val_loss: 4.5466 - val_acc: 0.1905
Epoch 490/500
 32/167 [====>.........................] - ETA: 0s - loss: 2.0692 - acc: 0.2500
Epoch 00490: saving model to checkpoints/checkpoint.ckpt

Consider using a TensorFlow optimizer from `tf.train`.
167/167 [==============================] - 0s 567us/sample - loss: 2.1856 - acc: 0.2455 - val_loss: 4.2134 - val